In [3]:
link_2016 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2016.csv"
link_2017 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2017.csv"
link_2018 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2018.csv"
link_2019 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-justicia-y-seguridad/delitos/delitos_2019.csv"
link_2020 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2020.csv"
link_2021 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-justicia-y-seguridad/delitos/delitos_2021.csv"

In [5]:
import pandas as pd

In [ ]:
pd.read_csv(link_2019, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2019.csv')
pd.read_csv(link_2018, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2018.csv')
pd.read_csv(link_2017, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2017.csv')
pd.read_csv(link_2016, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2016.csv')

In [15]:
pd.read_csv(link_2021, sep=';').loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']].to_csv('/data/delitos_2021.csv')
pd.read_csv(link_2020, sep=',', ).loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']].to_csv('./data/delitos_2020.csv')

In [39]:
links =[link_2016, link_2017, link_2018,link_2019]
links

['https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2016.csv',
 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2017.csv',
 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2018.csv',
 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-justicia-y-seguridad/delitos/delitos_2019.csv']

In [36]:
"link_2021".split("_")[1]

'2021'

In [62]:
def extract_load(links):
    dfs = []
    for link in links:
        anio = link.split("_")[1]
        #print(f"procesando datos de {anio}")
        data = pd.read_csv(link, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']]
        data["fecha"] = pd.to_datetime(data["fecha"])
        data.to_csv(f'./datalake/delitos_{anio}')
        dfs.append(data)
       # print(dfs)
    consolided = pd.concat(dfs, ignore_index=True)
    consolided.to_csv(f'./datalake/delitos_consolidado')
    return consolided

In [63]:
extract_load(links)

C:\Users\Cristian Salinas\AppData\Local\Temp\ipykernel_15588\1434259782.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["fecha"] = pd.to_datetime(data["fecha"])


,fecha,franja_horaria,tipo_delito,barrio,comuna,lat,long
0,2016-01-01,13,Robo (con violencia),Parque Avellaneda,9.0,-34.648730,-58.468536
1,2016-01-01,22,Robo (con violencia),Palermo,14.0,-34.576925,-58.407400
2,2016-01-01,1,Lesiones,Parque Patricios,4.0,-34.645197,-58.401099
3,2016-01-01,19,Hurto (sin violencia),Parque Avellaneda,9.0,-34.655760,-58.478681
4,2016-01-01,4,Robo (con violencia),Caballito,6.0,-34.624128,-58.436826
...,...,...,...,...,...,...,...
493359,2019-09-13,21,Robo (con violencia),Vélez Sársfield,10.0,-34.631202,-58.500760
493360,2019-09-13,20,Robo (con violencia),Vélez Sársfield,10.0,-34.624261,-58.495508
493361,2019-09-13,18,Robo (con violencia),Almagro,5.0,-34.611807,-58.421817
493362,2019-09-13,19,Robo (con violencia),Villa Santa Rita,11.0,-34.611037,-58.483334
